In [2]:
#Importing Necessary libiraries
import csv
import json
import pandas as pd
from pymongo import MongoClient
import pymongo
import ast
import time
from elasticsearch import Elasticsearch

In [172]:
#Read And Clean Jobs_Skills Data 
DF = pd.read_csv(r'E:\New Moonlit Stage\iNetworks Intern\Practical Task #1\jobs_skills.csv').drop(['_id'], axis=1)
DF = DF[(DF.skills != "['nan']") & (DF.title != "['nan']") & (DF.jobFunction != "['nan']") & (DF.industry != "['nan']")].drop_duplicates()
DF.skills = DF.skills.replace(r'[^\w\s ()]', ' ')
#print(DF.skills[150][0])
DF.shape

(5693, 4)

In [173]:
## Cleansing The Job Titles Function ##
def clean_job_titles_list(job_titles_list):
    all_job_titles = [ str(cleaned_title) for cleaned_title in job_titles_list ]
    all_job_titles = [ title.split(' - ')[0].strip().lower().split(',')[0].split('@')[0].strip().split(" at ")[0].strip().split('\\u')[0] for title in all_job_titles  if "-end" != title.lower()]
    all_job_titles = [ title.encode('ascii', 'ignore').decode("utf-8").replace("[fcis19]",'').split(" for ")[0].split(" in ")[0].strip() for title in all_job_titles ]
    all_job_titles = [ title.split("(")[0].strip() for title in all_job_titles]
    cleaned_job_titles = []
    for title in all_job_titles:
        if len(title.split("|")) >= 2:
            cleaned_job_titles.append(title.split("|")[0].strip() + " | " + title.split("|")[1].strip())
        else: 
            cleaned_job_titles.append(title.strip())
    return cleaned_job_titles
DF.title = clean_job_titles_list(DF.title)
Skills = [ast.literal_eval(skill) for skill in DF.skills]
DF['skills'] = Skills
#print(DF.skills[100] , DF.skills[100][0] , type(DF.skills[100]) , type(DF.skills[100][0]))

In [174]:
#Save Cleaned Version Of The Data
DF.to_csv('Cleaned_Version1.csv')

In [14]:
###############################
## Define Mongo Configrations 
###############################
Client = MongoClient('localhost',27017)
DB = Client.Jobs_Skills
Collection = DB.Coll
DB.segment.drop()
DB.list_collection_names()

['Coll']

In [3]:
# Converting CSV To JSON Format
DF = pd.read_csv('Cleaned_Version1.csv', index_col=0) 
JSON_ = json.loads(DF.to_json(orient='records'))
for document in JSON_:
    document.pop('Unnamed: 0.1', None)

In [4]:
print(JSON_[0])

{'title': 'property consultant', 'jobFunction': "['Sales/Retail']", 'industry': "['Real Estate/Property Management']", 'skills': "['Sales', 'Retail', 'Real Estate', 'Sales Target', 'Indoor Sales', 'Sales Skills', 'Property Sales']"}


In [15]:
# Insert JSON Documents
Collection.delete_many({})
Collection.insert_many(JSON_)
print(Collection.estimated_document_count())

5693


In [179]:
print(DB.list_collection_names())
print(Collection.inserted_ids)

['Coll']
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Jobs_Skills'), 'Coll.inserted_ids')


In [180]:
# Searching Before Indexing
start_time = time.time()
result2 = pd.DataFrame(list(Collection.find({"title" :"admin assistant"})))
end_time = time.time()
search_time_after_indexing = end_time - start_time
print ( 'Search Time after Indexing in second: ', search_time_after_indexing)
#print(len(result2))
print(Collection.count_documents({"title" :"admin assistant"}))

Search Time after Indexing in second:  0.022029399871826172
18


In [181]:
# Searching with Indexing 
Collection.create_index([("title", pymongo.TEXT)], name="search_index",default_language='english')
start_time = time.time()
result = pd.DataFrame(list(Collection.find({"title": "admin assistant"})))
end_time = time.time()
search_time_before_indexing = end_time - start_time
print ('Search Time before Indexing in second: ', search_time_before_indexing)
print(len(result))

Search Time before Indexing in second:  0.02403855323791504
18


In [185]:
#print(sorted(list(DB.collection.index_information())))
#print(DB.collection.getIndexes())

In [186]:
Collection.estimated_document_count()

5693

In [5]:
# Elastic Search Connection
ES = None
ES = Elasticsearch()
if ES.ping():
    print('Connected!')
else:
    print('Server could not connect!')

Connected!


In [6]:
ES.indices.delete(index='skill')
# Create Index
created = False
# index settings
settings = {
        "settings": {"number_of_shards": 1, "number_of_replicas": 0},
        "mappings": {"dynamic": "strict", "properties": {"title": {"type": "text"}, "jobFunction": {"type": "text"}, "industry": {"type": "text"},"skills": {"type": "text"},
                    "ingredients": {"type": "object","properties": {"step": {"type": "text"}}}, }}}   
try:
        if not ES.indices.exists('skill'):
            ES.indices.create(index='skill', ignore=400, body=settings)
            print('Index IS Created')
        created = True
except Exception as ex:
    print(str(ex))

Index IS Created


In [7]:
print(ES.indices.get_mapping('skill'))
JSON_[1]

{'skill': {'mappings': {'dynamic': 'strict', 'properties': {'industry': {'type': 'text'}, 'ingredients': {'properties': {'step': {'type': 'text'}}}, 'jobFunction': {'type': 'text'}, 'skills': {'type': 'text'}, 'title': {'type': 'text'}}}}}


{'title': 'sales representative',
 'jobFunction': "['Customer Service/Support', 'Sales/Retail']",
 'industry': "['Real Estate/Property Management']",
 'skills': "['Sales', 'Real Estate', 'Sales Target', 'Customer Service', 'Customer Care', 'Sales Skills']"}

In [8]:
# Store JSON File In The Index
try:
    for record in JSON_:
        outcome = ES.index(index='skill', body=record)
    print('DONE!')  
except Exception as ex:
    print('Error in indexing data')
    print(str(ex))

DONE!


In [9]:
print(len(JSON_))
print(outcome)

5693
{'_index': 'skill', '_type': '_doc', '_id': '46Q0f3ABWxGDBVvQN5EP', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5692, '_primary_term': 1}


In [10]:
# Search Query
Search = {'query': {'match': {'skills': 'Machine Learning'}}}
Result = ES.search(index='skill', body=Search)

In [11]:
# Get The Matched Documents Of The Query
before_time = time.time()
for Doc in Result['hits']['hits']:
    print(Doc['_source'])
after_time = time.time()
print("\nProcess time: ", after_time - before_time)

{'title': 'senior machine learning engineer', 'jobFunction': "['IT/Software Development', 'Engineering - Telecom/Technology']", 'industry': "['Computer Software']", 'skills': "['CNN', 'TensorFlow', 'Startup', 'Python', 'RNN', 'Computer Science', 'Machine Learing', 'Architecture', 'C++', 'Design', 'Theano', 'NLP', 'Engineering', 'Machine Learning', 'Statistics']"}
{'title': 'machine learning engineer', 'jobFunction': "['IT/Software Development', 'Engineering - Telecom/Technology']", 'industry': "['Music', 'Computer Software', 'Computer and Network Security']", 'skills': "['Python', 'Software Development', 'Computer Science', 'NLP', 'Information Technology (IT)', 'Machine Learning', 'SQL', 'Java']"}
{'title': 'senior data scientist', 'jobFunction': "['IT/Software Development', 'Engineering - Telecom/Technology']", 'industry': "['Engineering Services']", 'skills': "['PowerBI', 'Agile', 'Python', 'C#', 'SAS', 'Git', 'C++', 'Oracle', 'PySpark', 'Information Technology (IT)', 'Machine Learni

In [162]:
#ES.indices.delete(index='jobs')

In [12]:
# delete index if exists
if ES.indices.exists("jobs"):
    ES.indices.delete(index="jobs")
# index settings
settings = {"settings": {"number_of_shards": 1,"number_of_replicas": 0},
            "mappings": {"properties": {'title': {'type': 'text'},'jobFunction' : {'type': 'text'},'industry' : {'type': 'text'},'skills':{'type': 'text'},'url':{'type': 'text'}}}}
# create index
ES.indices.create(index="jobs", ignore=400, body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'jobs'}

In [16]:
# Import Data from MongoDB Into Elastic 
for index,job in enumerate(Collection.find().limit(30)):
    del job['_id']
    Out = ES.index(index='jobs', id=index, body=job)
print('DONE!')
print(Out)

DONE!
{'_index': 'jobs', '_type': '_doc', '_id': '29', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 29, '_primary_term': 1}


In [17]:
start_time = time.time()
res = ES.get(index='jobs', id=14)
print(res['_source'])
end_time = time.time()
print("\nProcess time: ", end_time - start_time)

{'title': 'administrator & document controller', 'jobFunction': "['Administration']", 'industry': "['Real Estate/Property Management']", 'skills': "['Organizing', 'Printing', 'Scanning', 'Copying', 'Document Control', 'Administration']"}

Process time:  0.5488042831420898
